In [11]:
from llama_index.llms.openai import OpenAI
import os
import nest_asyncio
import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        "Enter API key for OpenAI: ")

nest_asyncio.apply()

llm = OpenAI(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [12]:

from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

summarization_agent_system_message = """
You are an Product specification summarizer who can summarize a product specification.
For the input provided, create a summary with less than 50 words.

Ensure that the summary focuses on performance specifications
and safety features.
"""

summarize_agent = FunctionAgent(
    system_prompt=(summarization_agent_system_message),
    llm=llm,
    tools=[],
    verbose=True,
    initial_state={"summary": None},
)

review_agent = FunctionAgent(
    system_prompt=(
        "You are the ReviewAgent that can review the summarization and provide feedback. "
        "Your review should either approve the current summarization or request changes to be implemented."
        "You need to ensure the summary focuses on performance specifications and safety features"
    ),
    llm=llm,
    tools=[],
    verbose=True
)

In [13]:
from llama_index.core.workflow import Context
from pypdf import PdfReader
from llama_index.core.agent.workflow import AgentStream, ToolCallResult
import asyncio

async def call_summarize_agent(text: str) -> str:
    """Useful for summarizing information found in PDF files."""
    result = asyncio.run(summarize_agent.run(
        user_msg=f"Summarize the following text: {text}"
    ))

    return str(result)


async def call_summarize_agent_with_review(text: str, summary: str, review: str) -> str:
    """Useful for updating summarized text based on a review.
    accepts the original text, the summary and the review as parameters."""
    result = asyncio.run(summarize_agent.run(
        user_msg=f"""update the summary based on the review.
        original_text: {text}

        review: {review}

        summary: {summary}"""
    ))

    return str(result)


async def call_review_agent(summary: str) -> str:
    """Useful for reviewing the summarized text and providing feedback."""
    result = asyncio.run(review_agent.run(
        user_msg=f"Review the following summary: {summary}"
    ))

    return result


def too_long(text: str, max_length: int) -> bool:
    """Useful for evaluating if a text is longer the expected.
    This tool accepts as input the text and the maximum length
    and returns True in case the text is too long and false in
    case the text length is ok"""
    return len(text.split()) > max_length


def read_pdf(path: str) -> str:
    """This tool allows reading the content of a PDF file from the local file system"""
    content = ''
    reader = PdfReader(path)
    for page in range(len(reader.pages)):
        content += '\n' + reader.pages[page].extract_text()

    return content


In [14]:

from llama_index.core.tools import FunctionTool

read_pdf_tool = FunctionTool.from_defaults(read_pdf)
too_long_tool = FunctionTool.from_defaults(too_long)
call_summarize_agent_tool = FunctionTool.from_defaults(call_summarize_agent)
call_summarize_agent_with_review_tool = FunctionTool.from_defaults(call_summarize_agent_with_review)
call_review_agent_tool = FunctionTool.from_defaults(call_review_agent)


summary_agent = ReActAgent(
    tools=[read_pdf_tool, too_long_tool,
           call_summarize_agent_tool, call_review_agent_tool, call_summarize_agent_with_review_tool],
    llm=llm,
    verbose=True  # Set verbose for detailed logs
)

In [15]:

ctx = Context(summary_agent)
handler = summary_agent.run("Summarize the text in '/home/eshulman/git/AI/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/datasets/EcoSprint_Specification_Document.pdf' summary should not exceed 50 words. Please also make sure to review it properly.", ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Thought: The current language of the user is: English. I need to use a tool to help me summarize the PDF document.
Action: read_pdf
Action Input: {"path": "/home/eshulman/git/AI/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/datasets/EcoSprint_Specification_Document.pdf"}Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult

Call read_pdf with {'path': '/home/eshulman/git/AI/hands-on-agentic-ai-building-ai-agents-with-llamaindex-3962125/datasets/EcoSprint_Specification_Document.pdf'}
Returned: 
EcoSprint SpecificationDocument
1.Overview● TheEcoSprint isarevolutionaryelectricvehicle(EV)designedforefficiencyandperformance. Withitssleekdesignandstate-of-the-art technology, theEcoSprint